In [8]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import load_model
from sklearn.model_selection import train_test_split
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import io
from skimage.color import rgba2rgb
from skimage.color import gray2rgb
import numpy as np
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet_v2 import MobileNetV2


In [9]:
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return 1-f1_val

def focal_loss(y_true, y_pred):
    gamma = 2.0
    alpha = 0.25
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
os.chdir('/home/arkadiusz/Dokumenty/Dane_AI2020/skyhacks_hackathon_dataset/training_images')

In [17]:
os.listdir()

['630421.jpg',
 '696395.jpg',
 '687285.jpg',
 '689636.jpg',
 '685227.jpg',
 '694849.jpg',
 '685188.jpg',
 '633853.jpg',
 '687120.jpg',
 '689683.jpg',
 '631561.jpg',
 '687139.jpg',
 '681058.jpg',
 '689157.jpg',
 '690377.jpg',
 '683232.jpg',
 '679828.jpg',
 '694044.jpg',
 '710411.jpg',
 '685412.jpg',
 '684543.JPG',
 '681012.jpg',
 '629938.jpg',
 '684613.jpg',
 '681518.jpg',
 '684049.jpg',
 '685426.jpg',
 '697637.jpg',
 '693375.jpg',
 '687765.jpg',
 '682069.jpg',
 '678853.jpg',
 '684697.jpg',
 '694871.jpg',
 '633839.JPG',
 '695570.jpg',
 '680423.jpg',
 '682492.jpg',
 '687118.jpg',
 '679499.jpg',
 '692029.jpg',
 '628951.JPG',
 '683373.jpg',
 '631438.jpg',
 '688680.jpg',
 '694507.JPG',
 '690081.jpg',
 '683491.jpg',
 '691472.jpg',
 '693987.jpg',
 '628551.jpg',
 '686103.jpg',
 '690863.jpg',
 '678679.jpg',
 '631030.jpg',
 '629554.JPG',
 '710251.jpg',
 '693082.jpg',
 '681245.jpg',
 '690677.jpg',
 '629483.jpg',
 '628096.jpg',
 '697772.jpg',
 '688689.jpg',
 '687371.jpg',
 '697414.JPG',
 '688193.j

In [18]:
train_labels = pd.read_csv('/home/arkadiusz/Dokumenty/Dane_AI2020/skyhacks_hackathon_dataset/training_labels.csv')
#ściezka do folderu ze zdjęciami
filenames = os.listdir()

In [19]:
train_labels.set_index('Name',inplace=True)

In [21]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))
 
# Driver Code
li1 = [10, 15, 20, 25, 30, 35, 40]
li2 = [25, 40, 35]
print(Diff(li1, li2))

[10, 20, 30, 15]


In [22]:
small_object = ['Animals','Bench','Cross', 'Food', 'Footpath', 'Furniture', 'Monument','Motor vehicle','Person','Plants','Road', 'Rocks','Stairs','Trees','Windows']
train_labels_SO = train_labels[small_object]

big_object = Diff(train_labels.columns, small_object)
train_labels_BO = train_labels[big_object]

In [25]:
train_labels_BO

,Cultural institution,Castle,Lake,Mine,Watercraft,Building,Sports facility,Grass,Open-air museum,City,...,Mountains,Graveyard,Park,Forest,Cave,Amusement park,Reservoir,Snow,Church,Sport
Name,,,,,,,,,,,,,,,,,,,,,
625433.jpg,False,False,False,False,False,False,True,True,False,False,...,False,False,False,True,False,False,True,False,False,True
626248.jpg,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
626250.jpg,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
626255.jpg,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
626273.jpg,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
626303.jpg,False,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True
626312.jpg,False,False,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
626313.jpg,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
626349.JPG,False,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
from sklearn.utils import shuffle

def shuffle(filename, label, random_state = 2020):
    filenames_shuffled, labels_shuffled = shuffle(filename, label, random_state)
    #Do rozważenia w razie czego
    # saving the shuffled file.
    # you can load them later using np.load().
    #np.save('labels_shuffled.npy', labels_shuffled)
    #np.save('filenames_shuffled.npy', filenames_shuffled)
    return filenames_shuffled, labels_shuffled

In [6]:
random  = 1515
file_shuffled, BO_shuffled = shuffle(filename, train_labels_BO, random)
file_shuffled, SO_shuffled = shuffle(filename, train_labels_SO, random)
file_shuffled, shuffled = shuffle(filename, train_labels, random)

In [11]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
model_Mobile1 = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
model_Mobile2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
model_Resnet1 = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
model_Resnet2 = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)

In [2]:
model_Densenet1 = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
model_Densenet2 = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

NameError: name 'DenseNet121' is not defined

ValueError: list.remove(x): x not in list

In [12]:
def final_model(premodel, shape):
    x = premodel.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    stuff = Dense(shape, activation='sigmoid')(x)

    model = Model(inputs=premodel.input, outputs=[stuff])

    for layer in premodel.layers:
        layer.trainable = False
    return (model)

In [13]:
#chyba niepotrzebne
labels = train_labels.columns
targets = train_labels.to_numpy()

labels_SO = train_labels_SO.columns
targets_SO = train_labels_SO.to_numpy()


labels_BO = train_labels_BO.columns
targets_BO = train_labels_BO.to_numpy()

In [14]:
#niepotrzebne
imgset = []

for name in train_labels.index:
    try:
      image = io.imread(name)/255
    except:
      try:
        image = io.imread(name.lower())/255
      except:
        try:
          image = io.imread(name.upper())/255
        except:
          pass
    
    image = resize(image,input_shape)
    imgset.append(image)
    if len(imgset)%500 == 0:
      print(len(imgset))
    
imgset = np.array([np.array(imgset[i]) for i in range(len(imgset))])

500
1000
1500
2000
2500
3000
3500
4000


In [16]:
X_train, X_test, y_train, y_test = train_test_split(filename_shuffled, shuffled, test_size=0.2, random_state = 42)
X_SO_train, X_SO_test, y_SO_train, y_SO_test = train_test_split(filename_shuffled, SO_shuffled, test_size=0.2, random_state = 42)
X_BO_train, X_BO_test, y_BO_train, y_BO_test = train_test_split(filename_shuffled, BO_shuffled, test_size=0.2, random_state = 42)

In [ ]:
class My_Custom_Generator(keras.utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    return np.array([
            resize(imread('/content/all_images/' + str(file_name)), (80, 80, 3))
               for file_name in batch_x])/255.0, np.array(batch_y)

In [ ]:
batch_size = 32

my_training_batch_generator = My_Custom_Generator(X_train, y_train, batch_size)
my_test_batch_generator = My_Custom_Generator(X_test, y_test, batch_size)

#i analogicznie dla mniejszych modeli

In [18]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 97, 97, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________________

In [22]:
model.losses

[]

In [23]:
EarlyStop = EarlyStopping(monitor= "val_dense_6_f1", patience = 5, restore_best_weights=True)

losses = {
	"dense_1": "categorical_crossentropy",
}

model.compile(optimizer='adam', loss=losses, metrics = [f1])

In [24]:
model.fit_generator(generator = my_training_batch_generator,
                    steps_per_epoch = int(3800 // batch_size) #dobrać by rozmiar się zgadzał, 
                    epochs=12, callbacks = [EarlyStop],
                    validation_data = my_test_batch_generator,
                    validation_steps = int(950//batch_size) #dobrać by rozmiar się zgadzał,   
#model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=12, callbacks = [EarlyStop], batch_size = 16)  

Epoch 1/12
208/208 [==============================] - 30s 142ms/step - loss: 11.0220 - f1: 0.4676 - val_loss: 10.7223 - val_f1: 0.4208
Epoch 2/12
208/208 [==============================] - 31s 149ms/step - loss: 9.5533 - f1: 0.3518 - val_loss: 10.8150 - val_f1: 0.4079
Epoch 3/12
208/208 [==============================] - 29s 138ms/step - loss: 8.7707 - f1: 0.3066 - val_loss: 10.7350 - val_f1: 0.4034
Epoch 4/12
208/208 [==============================] - 29s 137ms/step - loss: 8.1546 - f1: 0.2639 - val_loss: 11.1804 - val_f1: 0.4081
Epoch 5/12
208/208 [==============================] - 29s 139ms/step - loss: 7.6886 - f1: 0.2187 - val_loss: 11.5369 - val_f1: 0.4181
Epoch 6/12
208/208 [==============================] - 29s 137ms/step - loss: 7.3168 - f1: 0.1792 - val_loss: 11.8983 - val_f1: 0.4151
Epoch 7/12
208/208 [==============================] - 29s 138ms/step - loss: 7.0921 - f1: 0.1455 - val_loss: 12.3914 - val_f1: 0.4217
Epoch 8/12
208/208 [==============================] - 29s 138

In [26]:
model.save(os.getcwd())

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Skyhacks 2020/SkyHacks2020/skyhacks_hackathon_dataset/training_images/assets


In [7]:
a = np.array([1,2,3,4,5,6])
a[[1,3]] = [0, 0]
a

array([1, 0, 3, 0, 5, 6])

In [ ]:
def final_results(pred1, pred2):
    result = np.empty([pred1.shape[0], pred1.shape[1]+pred2.shape[1]])
    for i in range(pred1.shape[0]):
        result[i,[0,3,4,5,6,7,9,12,14,15,16,17,18,21,22,23,24,27,31,32,33,36]] = pred1[i]
        result[i,[1,2,8,10,11,13,19,20,25,26,28,29,30,34,35,38]]
    return result
        